In [1]:
%cd ..

/home/eli/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='omniglot_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-3,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 50,
    "cooldown": 25,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader)

In [12]:
trainer.train()

Train Epoch: 1 [512/17352 (3%)] Loss: 1552.681641
Train Epoch: 1 [9592/17352 (55%)] Loss: 988.651357
Train Epoch: 1 [16939/17352 (98%)] Loss: 889.311752
    epoch          : 1
    loss           : 1044.2989977842685
    val_loss       : 889.4338993343251
    val_log_likelihood: -800.0936571734068
    val_log_marginal: -880.3878252848282
Train Epoch: 2 [512/17352 (3%)] Loss: 973.112305
Train Epoch: 2 [9476/17352 (55%)] Loss: 820.424800
Train Epoch: 2 [16883/17352 (97%)] Loss: 778.588976
    epoch          : 2
    loss           : 830.7503319373307
    val_loss       : 792.3946961293615
    val_log_likelihood: -744.1384637221156
    val_log_marginal: -785.3045041801646
Train Epoch: 3 [512/17352 (3%)] Loss: 787.868774
Train Epoch: 3 [10373/17352 (60%)] Loss: 753.989139
Train Epoch: 3 [16883/17352 (97%)] Loss: 745.425087
    epoch          : 3
    loss           : 764.7033451544048
    val_loss       : 744.6411603976234
    val_log_likelihood: -710.7182737689758
    val_log_marginal: -741.